In [ ]:
# Coder_Hussam Qassim

# Import the necessary libraries 
import tensorflow as tf 
from tensorflow.contrib.layers import fully_connected
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

# Define the RNN parameters
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

# Create the inpute and lable data 
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

# Create the graph on 1 input layer and 2 hidden layers and one output layer 
with tf.name_scope("RNN"):
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, x, dtype=tf.float32)
    logits = fully_connected(states, n_outputs, activation_fn=None)
    
# Create the cost function     
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

# Craete the optimizer 
learning_rate = 0.001
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op	= optimizer.minimize(loss)
    
# Evaluate the NN     
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
# Initialize the variables     
init = tf.global_variables_initializer()

# Initialize the saver for save the model
saver = tf.train.Saver()

# Fetch the data 
mnist = input_data.read_data_sets("data/")
x_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

# define the number of the epochs and the size of the batch
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            x_batch = x_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={x: x_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={x: x_test, y: y_test })
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "data/my_model_final.ckpt")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
0 Train accuracy: 0.946667 Test accuracy: 0.9161
1 Train accuracy: 0.986667 Test accuracy: 0.9542


In [ ]:
# Using the Neural Network
with tf.Session() as sess:
    saver.restore(sess,	"data/my_model_final.ckpt")
    x_new_scaled = [...] # some new images (scaled from 0 to 1)
    z = logits.eval(feed_dict={x: x_new_scaled})
    y_pred = np.argmax(z, axis=1)